In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import load_node
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job import JobCalculation

import numpy as np
import scipy.constants as const
import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import re
import gzip
import matplotlib.pyplot as plt
from collections import OrderedDict
import urlparse
import io

import matplotlib.pyplot as plt

import zipfile
import io
from IPython.display import FileLink, FileLinks
from base64 import b64encode
import StringIO

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
def get_calc_by_label(workcalc, label):
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
    qb.append(JobCalculation, output_of=WorkCalculation, filters={'label':label})
    assert qb.count() == 1
    calc = qb.first()[0]
    assert(calc.get_state() == 'FINISHED')
    return calc

In [ ]:
def load_afm_pp_data(afm_pp_calc):
    df_path = afm_pp_calc.out.retrieved.get_abs_path('df.npy')
    df_vec_path = afm_pp_calc.out.retrieved.get_abs_path('df_vec.npy')
    
    df_data = np.load(df_path)
    df_vec_data = np.load(df_vec_path)
    
    x_arr = df_vec_data[0, 0] + np.linspace(0.0, df_vec_data[1, 0], df_data.shape[2])
    y_arr = df_vec_data[0, 1] + np.linspace(0.0, df_vec_data[2, 1], df_data.shape[1])

    x_grid, y_grid = np.meshgrid(x_arr, y_arr)
    
    return x_grid, y_grid, df_data

In [ ]:
def make_afm_pic(fig, ax, data, i_z, title, extent):
    im = ax.imshow(data[2][i_z, :, :], origin='lower', cmap='gray', interpolation='bicubic', extent=extent)
    #pcm = ax.pcolormesh(data[0], data[1], data[2][i_z, :, :], cmap='gray', antialiased=True)
    ax.set_title(title)
    fig.colorbar(im, ax=ax)
    ax.axis('scaled')
    ax.set_xlabel(r"Tip$_x$ [$\AA$]")
    ax.set_ylabel(r"Tip$_y$ [$\AA$]")

In [ ]:
data_pp = None
data_2pp = None
h0 = None
dz = None
extent = None
figsize = None

def load_pk(b):
    global data_pp, data_2pp, h0, dz, extent, figsize
    try:
        workcalc = load_node(pk=pk_select.value)
        afm_pp_calc = get_calc_by_label(workcalc, 'afm_pp')
        afm_2pp_calc = get_calc_by_label(workcalc, 'afm_2pp')
    except:
        print("Incorrect pk.")
        return
    
    # --------------------
    # Clear last run...
    mk_zip_btn.disabled=True
    with html_link_out:
        clear_output()
    zip_progress.value=0.0
    # --------------------
    
    scan_start_z = workcalc.inp.afm_pp_params.dict.scanMin[2]
    ampl = workcalc.inp.afm_pp_params.dict.Amplitude
    dz = workcalc.inp.afm_pp_params.dict.scanStep[2]
    h0 = scan_start_z + ampl/2.0
    
    data_pp = load_afm_pp_data(afm_pp_calc)
    data_2pp = load_afm_pp_data(afm_2pp_calc)
    
    extent = [data_pp[0][0, 0], data_pp[0][0, -1], data_pp[1][0, 0], data_pp[1][-1, 0]]
    fig_y_size = 6.0
    figsize = ( (extent[1] - extent[0]) / (extent[3] - extent[2]) * fig_y_size * 1.2, fig_y_size)
    
    with afm_out:
        clear_output()
        for i_z in range(data_pp[2].shape[0]):

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(figsize[0]*2,figsize[1]))
            make_afm_pic(fig, ax1, data_pp, i_z, r"PP Tip$_z = %.2f\ \AA$" % (h0 + i_z*dz), extent)
            make_afm_pic(fig, ax2, data_2pp, i_z, r"2PP Tip$_z = %.2f\ \AA$" % (h0 + i_z*dz), extent)
            plt.show()
    
    mk_zip_btn.disabled = False
    
pk_select = ipw.IntText(value=0, description='pk')

load_pk_btn = ipw.Button(description='Load pk')
load_pk_btn.on_click(load_pk)

afm_out = ipw.Output()

In [ ]:
def create_zip_dl_link(b):
    
    mk_zip_btn.disabled = True
        
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, "w", zipfile.ZIP_DEFLATED, False) as zip_file:
        
        for i_z in range(data_pp[2].shape[0]):
            
            zip_progress.value = i_z/float(data_pp[2].shape[0]-1)
            
            imgdata = StringIO.StringIO()
            fig = plt.figure(figsize=figsize)
            make_afm_pic(fig, plt.gca(), data_pp, i_z, r"PP Tip$_z = %.2f\ \AA$" % (h0 + i_z*dz), extent)
            fig.savefig(imgdata, format='png', dpi=200, bbox_inches='tight')
            zip_file.writestr("pp/pp_%02d.png" % i_z, imgdata.getvalue())
            plt.close()
            
            imgdata = StringIO.StringIO()
            fig = plt.figure(figsize=figsize)
            make_afm_pic(fig, plt.gca(), data_2pp, i_z, r"2PP Tip$_z = %.2f\ \AA$" % (h0 + i_z*dz), extent)
            fig.savefig(imgdata, format='png', dpi=200, bbox_inches='tight')
            zip_file.writestr("pp2/pp2_%02d.png" % i_z, imgdata.getvalue())
            plt.close()
    
    filename = "afm_%d.zip"%pk_select.value
    
    # Empty the /tmp folder...
    ! rm -rf tmp && mkdir tmp
    
    with open('tmp/'+filename, 'wb') as f:
        f.write(zip_buffer.getvalue())
    
    with html_link_out:
        display(HTML('<a href="tmp/%s">download zip</a>' % filename))
    
    # The following doesn't work in CHROME, which has a 2MB limit
    # Works in firefox though...
    
    #zip_64encode = b64encode(zip_buffer.getvalue())
    #html = '<a download="{}" href="'.format(filename)
    #html += 'data:application/zip;name={};base64,{}"'.format(filename, zip_64encode)
    #html += ' target="_blank">Download AFM</a>'
    #display(HTML(html))
    
mk_zip_btn = ipw.Button(description='Make ZIP', disabled=True)
mk_zip_btn.on_click(create_zip_dl_link)

zip_progress = ipw.FloatProgress(
        value=0,
        min=0,
        max=1.0,
        description='progress:',
        bar_style='info',
        orientation='horizontal'
    )

html_link_out = ipw.Output()

display(pk_select, load_pk_btn, ipw.HBox([mk_zip_btn, zip_progress]), html_link_out,  afm_out)

In [ ]:
### Load the URL after everything is set up ###
try:
    url = urlparse.urlsplit(jupyter_notebook_url)
    pk_select.value = urlparse.parse_qs(url.query)['pk'][0]
    load_pk(0)
except:
    pass